In [72]:
import random
import string

In [73]:
class Steckerbrett:
    def __init__(self, wiring_pairs):
        '''
        Initialising the Steckerbrett with the letter pairs - just considering two pairs here
        
        wiring_pairs: list of tuples, for example, [('A', 'B'), ('C', 'D')]
        '''
        
        self.wiring = {}
        for a, b in wiring_pairs:
            self.wiring[a] = b
            self.wiring[b] = a  # creating a dictionary with the pairs

    def swap(self, letter):
        # swapping only those specific pairs
        return self.wiring.get(letter, letter)  # letter to swap

In [74]:
class Rotor:
    def __init__(self, wiring, notch, position=0):
        # Initialising a rotor
        # based on Caesar ciphers
        self.wiring = list(wiring)
        self.notch = notch  # position where the rotor causes the next rotor to advance
        self.position = position  # initial pos of rotor (0 by default)

    def forward(self, letter):
        # Passes the letter on to the next rotor
        index = (ord(letter) - ord('A') + self.position) % 26
        encoded_letter = self.wiring[index]
        return chr((ord(encoded_letter) - ord('A') - self.position) % 26 + ord('A'))  # reverse adjustment

    def back(self, letter):
        '''
        Passes the letter backward through the rotor - needed as the signal goes through the rotor
        in both directions during encryption and decryption
        '''
        
        index = (ord(letter) - ord('A') + self.position) % 26
        decoded_index = self.wiring.index(chr(index + ord('A')))
        return chr((decoded_index - self.position) % 26 + ord('A'))  # reverse adjustment

    def rotate(self):
        self.position = (self.position + 1) % 26  # rotor rotates by 1 position each time
        return self.position == self.notch  # True => the next rotor should rotate



In [75]:
class Reflector:
    def __init__(self, wiring):
        self.wiring = list(wiring) # reflector wiring

    def reflect(self, letter):
        index = ord(letter) - ord('A')
        return self.wiring[index]
        

In [76]:
class Enigma:
    def __init__(self, steckerbrett, rotors, reflector):
        self.steckerbrett = steckerbrett
        self.rotors = rotors
        self.reflector = reflector
        self.initial_pos = [r.position for r in rotors]

    def reset(self):
        for r, pos in zip(self.rotors, self.initial_pos):
            r.position = pos

    def encrypt(self, message):
        result = []
        for i in message:
            if i.isalpha():
                i = i.upper()
                # first passes through the steckerbrett
                i = self.steckerbrett.swap(i)

                # and then through the rotors
                for r in self.rotors:
                    i = r.forward(i)

                # reflection
                i = self.reflector.reflect(i)

                # back through the rotors
                for r in reversed(self.rotors):
                    i = r.back(i)

                # back through the steckerbrett again
                i = self.steckerbrett.swap(i)

                # rotation of rotors
                for r in self.rotors:
                    if not r.rotate():
                        break

            result.append(i)
        return ''.join(result)  # encrypted!!

    def decrypt(self, message):
        # resetting rotor positions before decrypting
        self.reset()
        return self.encrypt(message) # encrypting the letter as usual (works for decryption due to symmetry)


In [38]:
# IGNORE
# created just to generate rotor wiring settings
a = list(string.ascii_uppercase)
random.shuffle(a)
print(''.join(a))

VYTGQIPRMHSWFCDJEUKALOXZNB


In [124]:
# Configurations

steckerbrett_settings = [('Q', 'Z'), ('P', 'M')]
rotor1 = Rotor('TMFIAPLNRUBEGWJDKQXHCZVSOY', notch=16)
rotor2 = Rotor('AMFUSYCDWRZKLQGTJXVBOEIHPN', notch=4)
rotor3 = Rotor('VYTGQIPRMHSWFCDJEUKALOXZNB', notch=21)
ref = Reflector('YRUHQSLDPXNGOKMIEBFZCWVJAT') # using a standard ww2 setting instead as other wirings didn't work

steckerbrett = Steckerbrett(steckerbrett_settings)
enigma = Enigma(steckerbrett, [rotor1, rotor2, rotor3], ref)

### Encryption

In [132]:
# message = 'Das wetter in Berlin ist wahrscheinlich wunderbar heil Sahaana'
# enc = ''.join(enigma.encrypt(j) if j.isalpha() else j for j in message)
# print("Encrypted:", enc)

message = input('Enter your message:')
enc = enigma.encrypt(message)
print("Encrypted:", enc)

Enter your message: Das wetter in Berlin ist wahrscheinlich wunderbar heil Sahaana


Encrypted: LKW KGPCPI PQ FUJTFS YQF BVQFTFAUEWNNXV LLXXGCFEM ZZQI XFTFRID


### Decryption

In [133]:
dec = enigma.decrypt(enc)
print("Decrypted:", dec)

# resetting configurations -- SUPER IMPORTANT as decryption is sensitive to initial rotor settings
steckerbrett = Steckerbrett(steckerbrett_settings)
enigma = Enigma(steckerbrett, [rotor1, rotor2, rotor3], ref)

Decrypted: DAS WETTER IN BERLIN IST WAHRSCHEINLICH WUNDERBAR HEIL SAHAANA
